In [1]:
from dataLoader import getData
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/folds_phys_compare_pascal/'
u,i,o,p = getData(fold_dir,'abcd')

In [2]:
u.shape,i.shape,len(o),p.shape

((2500, 19734), (19734,), 19734, (839,))